# Let's start from modeling the problem

We will start by modeling the problem from the procedural solution just developed

```mermaid
classDiagram
    direction RL
    
    class User {
        -username: str
        -role: str
        -password: str
        +authenticate() bool
        +can_modify() bool
        +to_dict() Dict
    }
    
    class Document {
        -id: int
        -title: str
        -content: str
        -owner: User
        -visibility: str
        +change_visibility() bool
        +to_dict() Dict
    }
    
    class DocumentManager {
        -users: List[User]
        -documents: List[Document]
        -current_user: User
        +user_login() Tuple[User, bool]
        +get_document_index() int
        +retrieve_document() Document
        +change_document_visibility() bool
    }
    
    Document "1" *-- "1" User
    DocumentManager "1" *-- "*" User
    DocumentManager "1" *-- "*" Document
```

In [6]:
class User:
    """
    Simple User class.
    
    Attributes
    ----------
    username : str
        Unique identifier for the user
    role : str
        User's role in the system
    password : str
        Plain password
    
    Examples
    --------
    >>> user = User("alice", "user", "password123")
    >>> user.authenticate("password123")
    True
    """
    
    def __init__(self, username, role, password):
        self.username = username
        self.role = role
        self.password = password
    
    def authenticate(self, password: str) -> bool:
        """
        Authenticate user with password.
        
        Returns
        -------
        bool
            True if authentication successful, False otherwise
        """ 
        if not isinstance(password, str):
            return False
            
        if self.password == password:
            return True
        return False
        
    
    def can_modify(self, document: 'Document') -> bool:
        """Check if user can modify a document"""        
        # Administrators can modify everything
        if self.role == 'administrator':
            return True
        
        # Users can modify their own documents
        if document.owner == self:
            return True
        
        return False
    
    def to_dict(self):
        """Convert to dictionary for backward compatibility"""
        return {
            "username": self.username,
            "role": self.role,
            "password": self.password
        }
    
    def __str__(self):
        return f"User({self.username}, {self.role})"


In [7]:
class Document:
    """
    Document class representing a document in the system.
    
    Attributes
    ----------
    id : int
        Unique document identifier
    title : str
        Document title
    content : str
        Document content
    owner : User
        User who owns the document
    visibility : str
        Document visibility setting
    
    Examples
    --------
    >>> user = User("alice", "user", "password123")
    >>> doc = Document(1, "My Doc", "Content", user, "private)
    >>> doc.title
    'My Doc'
    """
    
    def __init__(self, doc_id, title, content, owner, visibility):
        self.id = doc_id
        self.title = title
        self.content = content
        self.owner = owner
        self.visibility = visibility
        
    def change_visibility(self, new_visibility, user: User) -> bool:
        """
        Change document visibility if user has permission.
        
        Parameters
        ----------
        new_visibility : str
            New visibility setting
        user : User
            User attempting the change
            
        Returns
        -------
        bool
            True if change successful, False otherwise
        """
        if user.can_modify(self):
            self.visibility = new_visibility
            return True
        return False
    
    def to_dict(self):
        """Convert to dictionary for backward compatibility"""
        return {
            "id": self.id,
            "title": self.title,
            "content": self.content,
            "owner": self.owner,
            "visibility": self.visibility
        }
    
    def __str__(self):
        return f"Document({self.id}, '{self.title}', {self.visibility})"


In [14]:
class DocumentManager:
    """
    Document manager that exactly mirrors the procedural functions.
    
    This class contains the same logic as the procedural version,
    but uses objects instead of dictionaries.
    """
    
    def __init__(self):
        # Initialize with the same data as procedural version
        self.users = [
            User("admin", "administrator", "admin"),
            User("alice", "user", "alice"), 
            User("bob", "user", "bob")
        ]
        
        self.documents = [
            Document(1, 
                     "Company Strategy", 
                     "Secret business plans...", 
                     self.users[0], 
                     "private"),
            Document(2, "Project Report", "Q1 project results...", self.users[1], "private"),
            Document(3, "Personal Notes", "My private thoughts...", self.users[1], "private"),
            Document(4, "Public Info", "Company public information...", self.users[0], "public")
        ]
        
        self.current_user = None
    
    def user_login(self, username, prompted_password):
        """
        Authenticate a user by username and password.
        
        This is an exact OOP translation of the procedural user_login function.
        
        Parameters
        ----------
        username : str
            The username to authenticate
        prompted_password : str
            The password provided by the user during login attempt
        
        Returns
        -------
        user_dict : User or None
            User object if username exists, None otherwise
        authenticated : bool
            True if authentication successful, False otherwise
        """
        for user in self.users:
            if user.username == username:
                if user.authenticate(prompted_password):
                    self.current_user = user
                    return user, True
                else:
                    return user, False
        return None, False
    
    def get_document_index(self, document_title):
        """
        Retrieve the document index by title if exists.
        
        This is an exact OOP translation of the procedural get_document_index function.
        
        Parameters
        ----------
        document_title : str
            The exact title of the document to retrieve

        Returns
        -------
        index: int or None
            Document index in the list, None otherwise
        """
        for index, doc in enumerate(self.documents):
            if doc.title == document_title:
                return index
        return None
    
    def retrieve_document(self, document_title, connected_user):
        """
        Retrieve a document by title if the user has permission to view it.
        
        This is an exact OOP translation of the procedural retrieve_document function.
        
        Parameters
        ----------
        document_title : str
            The exact title of the document to retrieve
        connected_user : User
            User object containing role and username attributes

        Returns
        -------
        document : Document or None
            Document object if found and user has permission, None otherwise
        """
        doc_index = self.get_document_index(document_title)
        if doc_index is None:
            return None
        doc = self.documents[doc_index]

        # Same permission logic as procedural version
        if connected_user.can_modify(doc) or doc.visibility == 'public':
            return doc
        return None
    
    def change_document_visibility(self, document_title, connected_user, new_visibility):
        """
        Change the visibility of a document if the user has permission.
        
        This is an exact OOP translation of the procedural change_document_visibility function.
        
        Parameters
        ----------
        document_title : str
            The exact title of the document to modify
        connected_user : User
            User object containing role and username attributes
        new_visibility : str
            The new visibility setting for the document

        Returns
        -------
        success : bool
            True if visibility was successfully changed, False otherwise
        """
        doc_index = self.get_document_index(document_title)
        if doc_index is None:
            return False
        doc = self.documents[doc_index]
        
        return doc.change_visibility(new_visibility, connected_user)



In [16]:



# =============================================================================
# TESTING - Exact same tests as procedural version
# =============================================================================

def run_oop_tests():
    """Run the same tests as the procedural version but with OOP"""
    print("=" * 70)
    print("OOP VERSION - EXACT MATCH TESTING")
    print("=" * 70)
    
    manager = DocumentManager()
    
    def print_test_result(test_name, expected, actual):
        """Helper function to print test results"""
        status = "PASS" if actual == expected else "FAIL"
        print(f"  {test_name}: {status}")
        print(f"    Expected: {expected}")
        print(f"    Actual: {actual}")
        if actual != expected:
            print(f"    ⚠️  MISMATCH!")
        print()
    
    # Test 1: User Login
    print("1. TESTING USER LOGIN")
    print("-" * 40)
    
    user, status = manager.user_login("alice", "alice")
    expected_user = manager.users[1]  # Alice is at index 1
    print_test_result("Alice login with correct password", (expected_user, True), (user, status))
    
    user, status = manager.user_login("alice", "wrongpassword")
    print_test_result("Alice login with wrong password", (expected_user, False), (user, status))
    
    user, status = manager.user_login("charlie", "anypassword")
    print_test_result("Non-existent user login", (None, False), (user, status))
    
    # Test 2: Document Retrieval  
    print("2. TESTING DOCUMENT RETRIEVAL")
    print("-" * 40)
    
    # Login users for testing
    admin_user, _ = manager.user_login("admin", "admin")
    alice_user, _ = manager.user_login("alice", "alice") 
    bob_user, _ = manager.user_login("bob", "bob")
    
    # Admin accessing private document
    doc = manager.retrieve_document("Company Strategy", admin_user)
    expected_doc = manager.documents[0]
    print_test_result("Admin accessing private doc", expected_doc, doc)
    
    # Owner accessing their own document
    doc = manager.retrieve_document("Project Report", alice_user)
    expected_doc = manager.documents[1]
    print_test_result("Owner accessing own private doc", expected_doc, doc)
    
    # Non-owner accessing private document
    doc = manager.retrieve_document("Project Report", bob_user)
    print_test_result("Non-owner accessing private doc", None, doc)
    
    # Any user accessing public document
    doc = manager.retrieve_document("Public Info", bob_user)
    expected_doc = manager.documents[3]
    print_test_result("Any user accessing public doc", expected_doc, doc)
    
    # Test 3: Document Visibility Changes
    print("3. TESTING DOCUMENT VISIBILITY CHANGES")
    print("-" * 40)
    
    # Admin changing any document
    result = manager.change_document_visibility("Project Report", admin_user, "public")
    print_test_result("Admin changing document visibility", True, result)
    
    # Verify the change
    doc = manager.retrieve_document("Project Report", bob_user)
    print_test_result("Bob can now see changed document", manager.documents[1], doc)
    
    # Restore original visibility
    manager.documents[1].visibility = "private"
    
    # Owner changing their own document
    result = manager.change_document_visibility("Project Report", alice_user, "public")
    print_test_result("Owner changing own document visibility", True, result)
    
    # Restore for further tests
    manager.documents[1].visibility = "private"
    
    # Test the bug: non-owner changing public document
    result = manager.change_document_visibility("Public Info", bob_user, "private")
    print_test_result("Non-owner changing public doc (BUG TEST)", False, result)
    
    # Demonstrate the bug explicitly
    print("4. BUG DEMONSTRATION")
    print("-" * 40)
    print("Current permission check in change_document_visibility:")
    print("if connected_user.role == 'administrator' or doc.visibility == 'public':")
    print("    has_right = True")
    print("This means ANY user can change visibility of PUBLIC documents!")
    print()
    
    # Make sure we have a public document
    manager.documents[3].visibility = 'public'
    
    # Bob (non-owner) tries to change a public document
    result = manager.change_document_visibility("Public Info", bob_user, "private")
    
    # Restore
    manager.documents[3].visibility = 'public'
    
    print("=" * 70)
    print("OOP TESTING COMPLETE")
    print("=" * 70)


if __name__ == "__main__":
    run_oop_tests()


OOP VERSION - EXACT MATCH TESTING
1. TESTING USER LOGIN
----------------------------------------
  Alice login with correct password: PASS
    Expected: (<__main__.User object at 0x7f90813fcc90>, True)
    Actual: (<__main__.User object at 0x7f90813fcc90>, True)

  Alice login with wrong password: PASS
    Expected: (<__main__.User object at 0x7f90813fcc90>, False)
    Actual: (<__main__.User object at 0x7f90813fcc90>, False)

  Non-existent user login: PASS
    Expected: (None, False)
    Actual: (None, False)

2. TESTING DOCUMENT RETRIEVAL
----------------------------------------
  Admin accessing private doc: PASS
    Expected: Document(1, 'Company Strategy', private)
    Actual: Document(1, 'Company Strategy', private)

  Owner accessing own private doc: PASS
    Expected: Document(2, 'Project Report', private)
    Actual: Document(2, 'Project Report', private)

  Non-owner accessing private doc: PASS
    Expected: None
    Actual: None

  Any user accessing public doc: PASS
    Ex